In [13]:
import pandas as pd
import requests
from datetime import datetime


pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 2)
UPDATE_PRODUCT_SPECIFICATION_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pvt/products/{product_id}/specification"
)
GET_SKU_BY_EAN_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pvt/sku/stockkeepingunitbyean/"
)
GET_FIELD_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pub/specification/fieldGet/"
)
GET_FIELD_VALUES_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pub/specification/fieldvalue/"
)
GET_CATEGORY_FIELD_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog_system/pub/specification/field/listByCategoryId/"
)
DELETE_PRODUCT_SPECIFICATIONS_ENDPOINT = (
    "https://challengerco.vtexcommercestable.com.br/api/catalog/pvt/product/{product_id}/specification"
)
API_APP_KEY_NAME = ""
API_APP_KEY_VALUE = ""
API_APP_TOKEN_NAME = ""
API_APP_TOKEN_VALUE = ""

In [14]:
"""
Leer excel products-master y convertirlo a csv
"""
products_master_excel_df = pd.read_excel("source-files/products-master.xlsx")
products_master_excel_df.to_csv("source-files/products-master.csv", encoding="utf-8")
products_master_csv_df = pd.read_csv("source-files/products-master.csv", encoding="utf-8")

In [15]:
"""
Filtrar solo por las columnas que pertenecen a una especificación
"""
valid_columns = [col for col in products_master_csv_df.columns if not str(col).startswith("Unnamed")]
products_master_new_df = products_master_csv_df[valid_columns]
products_master_new_df.drop(index=0, columns="ID CAMPO", inplace=True)
products_master_new_df

,135,133,134,136,138,137,139,141,140,240,242,241,142,143,144,145,146,EAN,167,168,169,166,209,224,19,20,21,"163, 25",35,"36, 37",38,39,40,"41, 42, 152",43,44,45,"98, 99, 101, 106, 221, 223",164,22,23,24,27,"28, 207",29,30,31,32,33,34,47,108,"173, 213, 252",214,215,"153, 155,156, 216, 225","212, 217",218,"158, 219","211, 149, 220",109,226,"110, 253",233,111,151,185,113,244,"114, 181, 208","119, 120","121, 179",182,126,"124, 174",125,183,"175, 127",128,"222, 238","130, 239",131,132,"246, 249",250,245,75,"76, 234, 235, 229","77, 230, 180","247, 248","78, 176",227,228,"82, 232",83,84,85,86,87,88,243
1,Eficiencia.,7707172626948-1.jpg,Quemadores italianos SABAF de alta eficiencia ...,Durabilidad.,7707172626948-2.jpg,Perillas metálicas que garantizan una mayor v...,Seguridad y estabilidad.,7707172626948-3.jpg,Parrillas en hierro fundido con soportes antid...,Praticidad.,7707172626948-4.jpg,Encendido electrónico incorporado que permite...,Como solicitar un servicio o instalacion___Com...,HE2750-01.jpg,7707172626948_manual_de_usuario.pdf,7707172626948_hoja_de_garantia.pdf,7707172626948_ficha_tecnica.pdf,7707172626948,"32+13,5+56,5 (Ancho x Alto x Profundidad)",7.2,6.5,30x52 (Ancho x Profundidad),"28x47,7 (Ancho x Profundidad)",NaN,NaN,NaN,NaN,NaN,1 año,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,287.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vidrio negro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural,NaN,NaN,2,"Un (1) Quemador Auxiliar (1,1KW)[3,95 MJ/h].\n...","(4,0 KW)[14,37 MJ/h].",Gas,Dos (2) De hierro fundido con soportes antides...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Multifunciones,7705191002712-1.jpg,Reinicio Automático\nFunción Turbo\nTemporizad...,Comodidad,7705191002712-2.jpg,4 velocidades de ventilación,Ambiente natural,7705191002712-3.jpg,Producción de oxígeno activo\nEliminación de h...,Más resistente,7705191002712-4.jpg,ABS alto brillo antirayas,Como solicitar un servicio o instalacion___Com...,HE2750-01.jpg,7705191002712_manual_de_usuario.pdf,7705191002712_hoja_de_garantia.pdf,7705191002712_ficha_tecnica.pdf,7705191002712,"52+88,5+45 (Ancho x Alto x Profundidad)",35,32,"48x79,5x40 (Ancho x Alto x Profundidad)",NaN,NaN,NaN,NaN,NaN,NaN,Compresor: 60 meses\nSobre los demás component...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kit de instalación en ventana de salida de air...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blanco.,Portátil,T1 (18˚ - 43˚C),115 / 60,12000 BTU/h (3200W) / 1 TON,NaN,12000


In [16]:
# Archivo de prueba con columnas filtradas
# products_master_new_df.to_excel("generated-files/products-master-just-fields-with-id.xlsx")

In [17]:
"""
Separar columnas de especificaciones con ids diferentes
"""
multiple_fields_columns_name = [col for col in products_master_new_df.columns if len(str(col).split(",")) > 1]
for column in multiple_fields_columns_name:
    multiple_fields_columns = [new_column for new_column in column.replace(" ", "").split(",")]
    for new_column in multiple_fields_columns:
        products_master_new_df[new_column] = products_master_new_df[column]
    products_master_new_df.drop(columns=[column], inplace=True)

In [18]:
# Archivo de prueba con una columna por id de especificación
products_master_new_df

,135,133,134,136,138,137,139,141,140,240,242,241,142,143,144,145,146,EAN,167,168,169,166,209,224,19,20,21,35,38,39,40,43,44,45,164,22,23,24,27,29,30,31,32,33,34,47,108,214,215,218,109,226,233,111,151,185,113,244,182,126,125,183,128,131,132,250,245,75,227,228,83,84,85,86,87,88,243,163,25,36,37,41,42,152,98,99,101,106,221,223,28,207,173,213,252,153,155,156,216,225,212,217,158,219,211,149,220,110,253,114,181,208,119,120,121,179,124,174,175,127,222,238,130,239,246,249,76,234,235,229,77,230,180,247,248,78,176,82,232
1,Eficiencia.,7707172626948-1.jpg,Quemadores italianos SABAF de alta eficiencia ...,Durabilidad.,7707172626948-2.jpg,Perillas metálicas que garantizan una mayor v...,Seguridad y estabilidad.,7707172626948-3.jpg,Parrillas en hierro fundido con soportes antid...,Praticidad.,7707172626948-4.jpg,Encendido electrónico incorporado que permite...,Como solicitar un servicio o instalacion___Com...,HE2750-01.jpg,7707172626948_manual_de_usuario.pdf,7707172626948_hoja_de_garantia.pdf,7707172626948_ficha_tecnica.pdf,7707172626948,"32+13,5+56,5 (Ancho x Alto x Profundidad)",7.2,6.5,30x52 (Ancho x Profundidad),"28x47,7 (Ancho x Profundidad)",NaN,NaN,NaN,NaN,1 año,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120,120,120,120,120,120,NaN,NaN,NaN,NaN,NaN,B,B,B,B,B,287.5,287.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vidrio negro,Vidrio negro,Vidrio negro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Natural,Natural,"Un (1) Quemador Auxiliar (1,1KW)[3,95 MJ/h].\n...","Un (1) Quemador Auxiliar (1,1KW)[3,95 MJ/h].\n...","Un (1) Quemador Auxiliar (1,1KW)[3,95 MJ/h].\n...","Un (1) Quemador Auxiliar (1,1KW)[3,95 MJ/h].\n...","(4,0 KW)[14,37 MJ/h].","(4,0 KW)[14,37 MJ/h].","(4,0 KW)[14,37 MJ/h].",Gas,Gas,Dos (2) De hierro fundido con soportes antides...,Dos (2) De hierro fundido con soportes antides...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Multifunciones,7705191002712-1.jpg,Reinicio Automático\nFunción Turbo\nTemporizad...,Comodidad,7705191002712-2.jpg,4 velocidades de ventilación,Ambiente natural,7705191002712-3.jpg,Producción de oxígeno activo\nEliminación de h...,Más resistente,7705191002712-4.jpg,ABS alto brillo antirayas,Como solicitar un servicio o instalacion___Com...,HE2750-01.jpg,7705191002712_manual_de_usuario.pdf,7705191002712_hoja_de_garantia.pdf,7705191002712_ficha_tecnica.pdf,7705191002712,"52+88,5+45 (Ancho x Alto x Profundidad)",35,32,"48x79,5x40 (Ancho x Alto x Profundidad)",NaN,NaN,NaN,NaN,NaN,Compresor: 60 meses\nSobre los demás component...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blanco.,Portátil,T1 (18˚ - 43˚C),115 / 60,12000 BTU/h (3200W) / 1 TON,NaN,12000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120,120,120,120,120,120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kit de instalación en ventana de salida de air...,Kit de instalación en ventana de salida de air...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
specifications_clean_excel_df = pd.read_excel("source-files/specifications-clean.xls")
specifications_clean_excel_df.to_csv("source-files/specifications-clean.csv", encoding="utf-8")
specifications_master_csv_df = pd.read_csv("source-files/specifications-clean.csv", encoding="utf-8")
specifications_master_csv_df.index.name = "index"

specifications_log_clean_excel_df = pd.read_excel("source-files/specifications-log-clean.xls")
specifications_log_clean_excel_df.to_csv("source-files/specifications-log-clean.csv", encoding="utf-8")
specifications_log_csv_df = pd.read_csv("source-files/specifications-log-clean.csv", encoding="utf-8")

In [20]:
api_request_headers = {
    "accept": "application/json",
    "content-type": "application/json",
    API_APP_KEY_NAME: API_APP_KEY_VALUE,
    API_APP_TOKEN_NAME: API_APP_TOKEN_VALUE,
}
for index, row in products_master_new_df.iterrows():
    try:
        specification_log_row = {
            "product_id": None,
            "ean": row['EAN'],
            "category_fields": None,
            "master_products_valid_fields": None,
            "missing_fields": None,
            "product_not_found": False,
            "json_decode_errors": [],
        }

        print("EAN", row['EAN'])
        sku = requests.request(
            "GET",
            f"{GET_SKU_BY_EAN_ENDPOINT}{row['EAN']}",
            headers=api_request_headers
        ).json()
        if isinstance(sku, str) and sku == "SKU not found.":
            specification_log_row["product_not_found"] = True
            continue

        

        # BORRAR SPECIFICATIONS
        delete_specifications = False

        if delete_specifications:
            requests.request(
                "DELETE",
                DELETE_PRODUCT_SPECIFICATIONS_ENDPOINT.replace("{product_id}", str(sku["ProductId"])),
                headers=api_request_headers
            )


        specification_log_row["product_id"] = sku["ProductId"]

        valid_fields = []
        for category in sku["ProductCategories"].keys():
            category_valid_fields = requests.request(
                "GET",
                f"{GET_CATEGORY_FIELD_ENDPOINT}{category}",
                headers=api_request_headers
            ).json()
            valid_fields += category_valid_fields
        valid_fields = [field["FieldId"] for field in valid_fields]
        valid_fields += [142, 35, 166, 167, 168, 169, 135, 134, 133, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 240, 242, 241]
        specification_log_row["category_fields"] = ", ".join([str(field) for field in valid_fields])
        print("valid_fields", valid_fields, len(valid_fields))
        
        set_valid_fields = set(valid_fields)
        set_master_products_fields = set([int(field_id) for field_id in row.keys() if field_id != "EAN"])

        specification_log_row["missing_fields"] = ", ".join([str(field) for field in set_valid_fields.difference(set_master_products_fields)])
        specification_log_row["master_products_valid_fields"] = ", ".join([str(field) for field in set_valid_fields.intersection(set_master_products_fields)])

        for field_id in row.keys(): 
            if field_id == "EAN":
                continue
            if int(field_id) in valid_fields:                
                print("field id", field_id)
                try:
                    field = {
                        "value": row[field_id],
                        "api_field": requests.request(
                            "GET",
                            f"{GET_FIELD_ENDPOINT}{field_id}",
                            headers=api_request_headers
                        ).json(),
                        "api_field_values": requests.request(
                            "GET",
                            f"{GET_FIELD_VALUES_ENDPOINT}{field_id}",
                            headers=api_request_headers
                        ).json(),
                    }
                except JSONDecodeError:
                    specification_log_row["json_decode_errors"].append("row[field_id]: " + row[field_id] + ", field_id:" + field_id)
                    continue
                

                specification_row = {
                    "_ProductId (Not changeable": sku["ProductId"],
                    "_ProductName (Not changeable": sku["ProductName"],
                    "FieldId (Not changeable": field_id,
                    "FieldName (Not changeable": field["api_field"]["Name"],
                    "FieldTypeName (Not changeable": field["api_field"]["FieldTypeName"],
                    "FieldValueId (Not changeable": ",".join([str(field_value["FieldValueId"]) for field_value in field["api_field_values"]]) if len(field["api_field_values"]) else "",
                    "FieldValueName (Not changeable": ",".join([field_value["Value"] for field_value in field["api_field_values"]]) if len(field["api_field_values"]) else "",
                    "SpecificationCode (Not changeable": "",
                    "SpecificationValue": field["value"],
                    "_ProductReferenceCodeId (Not changeable": sku["ProductRefId"],
                }
                # specifications_master_csv_df = specifications_master_csv_df.append(specification_row, ignore_index=True)
                specifications_clean_excel_df = specifications_clean_excel_df.append(specification_row, ignore_index=True)
        
                # UPDATE SPECIFICATIONS
                update_specifications = True
                body_update_field_value = [
                    {
                        "Value": [
                            field["value"]
                        ],
                        "Id": field_id,
                        "Name": field["api_field"]["Name"]
                    }
                ]
                print("update body value field", body_update_field_value)
                if update_specifications:
                    update_response = requests.request(
                        "POST",
                        UPDATE_PRODUCT_SPECIFICATION_ENDPOINT.replace("{product_id}", str(sku["ProductId"])),
                        headers=api_request_headers,
                        json=body_update_field_value
                    )
                    print(update_response)
        specification_log_row["json_decode_errors"] = ",".join(specification_log_row["json_decode_errors"])
        specifications_log_csv_df = specifications_log_csv_df.append(specification_log_row, ignore_index=True)
    
    
    except TypeError as e:
        print(e)

36
update body value field [{'Value': ['Durabilidad'], 'Id': '136', 'Name': 'título 2'}]
<Response [200]>
field id 138
update body value field [{'Value': ['7705191030623-2.jpg'], 'Id': '138', 'Name': 'imagen 2'}]
<Response [200]>
field id 137
update body value field [{'Value': ['Circuito de refrigeración en cobre. Sistema Golden fin: revestimiento anticorrosivo del condensador que prolonga la vida útil del aparato.'], 'Id': '137', 'Name': 'texto 2'}]
<Response [200]>
field id 139
update body value field [{'Value': ['Multifunciones'], 'Id': '139', 'Name': 'título 3'}]
<Response [200]>
field id 141
update body value field [{'Value': ['7705191030623-3.jpg'], 'Id': '141', 'Name': 'imagen 3'}]
<Response [200]>
field id 140
update body value field [{'Value': ['Encendido y apagado automático (timer).\nTemporizador. Modo  Sleep (dormir).\nFunción Strong.\nFunción Feeling (sensación).\nFunción Clean (limpieza de evaporador).'], 'Id': '140', 'Name': 'texto 3'}]
<Response [200]>
field id 240
upda

In [21]:
rows = len(specifications_master_csv_df)
page_limit= 998
start_row = 0
if rows > page_limit:
    pages = -(-rows // page_limit)
    for page in range(1, pages + 1):
        print(page)
        end_row = (page * page_limit)
        if end_row > rows:
            end_row = rows
        file_name = "specifications-master-" + str(page)
        specifications_master_csv_df.loc[start_row:end_row,:].to_csv("generated-files/" + file_name + ".csv", encoding="utf-8")
        specifications_master_excel_df = pd.read_csv("generated-files/" + file_name + ".csv", encoding="utf-8")
        valid_columns = [col for col in specifications_master_excel_df.columns if not str(col).startswith("Unnamed")]
        specifications_master_excel_df = specifications_master_excel_df[valid_columns]
        specifications_master_excel_df.to_excel("generated-files/" + file_name + ".xls", index=False)
        start_row = end_row

In [22]:
rows = len(specifications_clean_excel_df)
print(rows)
page_limit= 998
start_row = 0
if rows > page_limit:
    pages = -(-rows // page_limit)
    for page in range(1, pages + 1):
        print(page)
        end_row = (page * page_limit)
        if end_row > rows:
            end_row = rows
        file_name = "specifications-master-" + str(page)
        specifications_clean_excel_df.loc[start_row:end_row,:].to_excel("generated-files/" + file_name + ".xls", encoding="utf-8")
        start_row = end_row
else:
    specifications_clean_excel_df.to_excel("generated-files/specifications-master.xls", encoding="utf-8")

6366
1
2
3
4
5
6
7


In [23]:
specifications_clean_excel_df

,_ProductId (Not changeable,_ProductName (Not changeable,FieldId (Not changeable,FieldName (Not changeable,FieldTypeName (Not changeable,FieldValueId (Not changeable,FieldValueName (Not changeable,SpecificationCode (Not changeable,SpecificationValue,_ProductReferenceCodeId (Not changeable
0,1,Estufa de empotrar en vidrio templado de 2 pue...,135,título 1,Texto,,,,Eficiencia.,1.6730.20
...,...,...,...,...,...,...,...,...,...,...
6365,190,Aire Acondicionado portátil,120,Filtros,Texto,,,,NaN,11.122.11.51


In [24]:
now = str(datetime.now()).replace(" ", "-").replace(":", "_").split(".")[0]
specifications_log_csv_df.to_excel("generated-files/specifications-log-" + now + ".xls", index=False)

In [25]:
datetime.now()

datetime.datetime(2020, 11, 1, 1, 6, 19, 68233)